# Hyperparameter

Hyperparameter adalah parameter yag dapat disesuaikan untuk mengontrol proses pemgoptimalan model. Hyperparameter pada saat proses iterasi tidak akan diupdate atau sifatnya tetap.
- Epoch: proses berulang selama proses pelatihan, dimana 1 epoch = keseluruhan dataset telah dilatih.
- Batch jumlah sampel yang digunakan pada proses training.
- Learning Rate: kecepatan model dalam belajar. Nilai learning rate yang kecil menyebabkan proses belajar model menjadi lambat dan sebaliknya jika learning rate besar, maka proses belajar model menjadi cepat.

# Loss Function

Digunakan untuk mengukur ketidakmiripan hasil dari prediksi model dengan nilai aktual.

In [1]:
X_actual = 10
X_pred = 9.6

loss = X_actual - X_pred
print(f"Loss: {loss}")

Loss: 0.40000000000000036


Pada contoh diatas selisih antara *X_actual* dan *X_pred* disebut *loss* atau *error*. Semakin besar kerugiannya (*loss*) maka performa model masih kurang baik dan sebaliknya jika semakin kecil maka performa model semakin baik.

Pada Pytroch terdapat beberapa fungsi loss, berikut tautannya https://pytorch.org/docs/stable/nn.html#loss-functions

In [2]:
import torch
from torch import nn

torch.manual_seed(42)
y_pred = torch.randn(size=(10, ))
y_actual = torch.randn(size=(10, ))

mse_loss = nn.MSELoss()
print(f"MSE Loss: {mse_loss(y_pred, y_actual).item()}")

MSE Loss: 1.1729463338851929


C:\Users\62821\.conda\envs\dl_with_me\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Optimizer

- Digunakan untuk mengoptimalkan nilai-nilai dari parameter agar nilai kerugian (*loss*) semakin berkurang.
- Menggunakan Gradient Descent untuk melakukan optimasi parameter.
- Pada Pytroch htersedia beberapa optimizer yang dapat dilihat pada tautan https://pytorch.org/docs/stable/optim.html

In [3]:
from torch import nn, optim

# Build Model
model = nn.Sequential(
            # layer 1 (4 neurons) to layer 2 (2 neurons) with ReLU
            nn.Linear(4, 2),
            nn.ReLU(),
            
            # layer 2 (2 neurons) to layer 3 (1 neurons) with Sigmoid
            nn.Linear(2, 1),
            nn.Sigmoid(),
        )

optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)